# Entropy Vector for Hypergraphs

Implementation of the entropy vector $SE(H)$ from *"A New Entropy for Hypergraphs"*.

The entropy vector is a $2^m - 1$ dimensional descriptor that captures the fine structure of a hypergraph by computing Shannon entropy over all partial hypergraphs.

In [ ]:
import numpy as np
from itertools import combinations
from math import comb

## 1. Incidence Matrix

Construct an $m \times n$ incidence matrix $I$ where $I_{i,j} = 1$ if vertex $x_j \in e_i$.

In [ ]:
def build_incidence_matrix(hyperedges, num_vertices):
    """Build the m x n incidence matrix from a list of hyperedges.

    Args:
        hyperedges: list of sets/lists, each containing vertex indices.
        num_vertices: total number of vertices n.

    Returns:
        I: np.ndarray of shape (m, n).
    """
    m = len(hyperedges)
    I = np.zeros((m, num_vertices), dtype=np.float64)
    for i, edge in enumerate(hyperedges):
        for v in edge:
            I[i, v] = 1.0
    return I

## 2. Core Entropy Function $S(H)$

For a (partial) hypergraph with incidence matrix $I$:

1. Compute $L(H) = I \cdot I^T$ (the $m \times m$ intersection matrix)
2. Extract eigenvalues $\lambda_i$
3. Normalize: $\mu_i = \lambda_i / \mathrm{Tr}(L)$
4. Shannon entropy: $S(H) = -\sum \mu_i \log_2(\mu_i)$

In [ ]:
def compute_entropy(incidence_matrix):
    """Compute Shannon entropy S(H) from an incidence matrix.

    Args:
        incidence_matrix: np.ndarray of shape (m, n).

    Returns:
        entropy: float, the Shannon entropy in bits.
    """
    # L(H) = I * I^T, entry (i,j) = |e_i cap e_j|
    L = incidence_matrix @ incidence_matrix.T

    eigenvalues = np.linalg.eigvalsh(L)

    # Filter out zero/negative eigenvalues (numerical noise)
    eigenvalues = eigenvalues[eigenvalues > 1e-12]

    trace = eigenvalues.sum()
    if trace < 1e-12:
        return 0.0

    mu = eigenvalues / trace

    entropy = -np.sum(mu * np.log2(mu))
    return entropy

## 3. Entropy Vector $SE(H)$

Iterate over all $2^m - 1$ non-empty subsets of hyperedges, grouped by subset size. For each size $i$, collect entropies sorted in increasing order, then concatenate.

In [ ]:
def entropy_vector(hyperedges, num_vertices):
    """Compute the full entropy vector SE(H).

    Args:
        hyperedges: list of sets/lists of vertex indices.
        num_vertices: total number of vertices.

    Returns:
        se: np.ndarray of length 2^m - 1.
    """
    m = len(hyperedges)
    full_incidence = build_incidence_matrix(hyperedges, num_vertices)

    se = []
    for size in range(1, m + 1):
        entropies = []
        for subset_indices in combinations(range(m), size):
            # Extract rows for this partial hypergraph
            partial_incidence = full_incidence[list(subset_indices), :]
            entropies.append(compute_entropy(partial_incidence))
        # Sort in increasing order within each scale
        entropies.sort()
        se.extend(entropies)

    return np.array(se)

## Example

A small hypergraph with 6 vertices and 3 hyperedges:
- $e_0 = \{0, 1, 2\}$
- $e_1 = \{1, 2, 3\}$
- $e_2 = \{3, 4, 5\}$

With $m=3$ we get $2^3 - 1 = 7$ entries in the entropy vector.

In [ ]:
hyperedges = [
    {0, 1, 2},    # e0
    {3,4},    # e1
    {4, 5},    # e2
]
num_vertices = 6

# Build and inspect the incidence matrix
I = build_incidence_matrix(hyperedges, num_vertices)
print("Incidence matrix I (m=3, n=6):")
print(I)

# Intersection matrix L = I * I^T
L = I @ I.T
print("\nIntersection matrix L = I * I^T:")
print(L)
print(f"Diagonal entries (hyperedge sizes): {np.diag(L).astype(int)}")
print(f"|e0 ∩ e1| = {int(L[0,1])}, |e0 ∩ e2| = {int(L[0,2])}, |e1 ∩ e2| = {int(L[1,2])}")

In [ ]:
# Full hypergraph entropy
S_full = compute_entropy(I)
print(f"S(H) for full hypergraph: {S_full:.4f} bits")

# Entropy vector
se = entropy_vector(hyperedges, num_vertices)
print(f"\nEntropy vector SE(H) ({len(se)} entries):")

m = len(hyperedges)
idx = 0
for size in range(1, m + 1):
    count = comb(m, size)
    block = se[idx:idx + count]
    print(f"  Size {size} ({count} subsets): {np.round(block, 4)}")
    idx += count